# Compute annual timeseries & epoch means

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
from itertools import product

import cartopy
import cartopy.crs as ccrs
import cmocean
import data_collections as dc
import funnel
import intake
import matplotlib.pyplot as plt
import numpy as np
import operators as ops
import pop_tools
import util
import xarray as xr

/glade/work/mclong/miniconda3/envs/metabolic/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [3]:
# if False:
try:
    cluster
    client
except:
    cluster, client = util.get_ClusterClient(memory='25GB')
    cluster.scale(200)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/8787/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.60:32911,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mclong/calcs/proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [4]:
sub_spec = dict(
    name='drift-corrected',
    experiment=['20C', 'RCP85'],
    member_id=dc.ocean_bgc_member_ids,
)

catalog = funnel.to_intake_esm(agg_member_id=False).search(**sub_spec)
experiment_list = sorted(catalog.unique('experiment')['experiment']['values'])
member_id_list = sorted(catalog.unique('member_id')['member_id']['values'])

In [11]:
def vol_weighted_mean(ds):
    vol_mask = ops.pop_ocean_volume(ds)
    return ds.weighted(vol_mask).mean(['z_t', 'nlat', 'nlon']).drop(['REGION_MASK'])


def vol_weighted_mean_blw200m(ds):
    vol_mask = ops.pop_ocean_volume(ds)
    vol_mask = vol_mask.where(vol_mask.z_t > 200e2).fillna(0.0)
    return ds.weighted(vol_mask).mean(['z_t', 'nlat', 'nlon']).drop(['REGION_MASK'])


operations = {
    "ann": dict(
        func=ops.resample_ann,
        add_ops=["resample_ann"],
        dep_name="drift-corrected",
    ),
    "ann.ts-glb": dict(
        func=vol_weighted_mean,
        add_ops=["resample_ann", "vol-weighted-mean"],
        dep_name="drift-corrected.ann",
    ),
    "ann.ts-glb-blw200m": dict(
        func=vol_weighted_mean_blw200m,
        add_ops=["resample_ann", "vol-weighted-mean-blw200m"],
        dep_name="drift-corrected.ann",
    ),
}

ops_ordered = ['ann'] + [k for k in operations.keys() if k != 'ann']
ops_ordered

['ann', 'ann.ts-glb', 'ann.ts-glb-blw200m']

In [12]:
clobber = False
stream = 'pop.h'
component = 'ocn'

variable_list = ['TEMP', 'pO2']

for operation in ops_ordered:
    info = operations[operation]

    dep_name = info['dep_name']
    add_ops = info['add_ops']
    func = info['func']

    catalog = funnel.to_intake_esm(agg_member_id=False)

    for experiment, member_id, variable in product(experiment_list, member_id_list, variable_list):
        # check for existing cache file
        asset = dc.fnl_gen_cache_file_name(
            experiment, component, stream, member_id, variable, f'drift-corrected.{operation}'
        )

        if clobber and os.path.exists(asset):
            print(f'removing: {asset}')
            shutil.rmtree(asset)

        if os.path.exists(asset):
            # print(f'exists: {asset}')
            continue

        with util.timer(f'{operation}.{member_id}.{variable}'):
            cat = catalog.search(
                name=dep_name,
                experiment=experiment,
                member_id=member_id,
                stream=stream,
                component=component,
                variable=variable,
            )

            dset = cat.to_dataset_dict()
            assert len(dset.keys()) == 1
            _, ds = dset.popitem()
            ds['REGION_MASK'] = pop_tools.get_grid('POP_gx1v6')['REGION_MASK']

            dso = func(ds)

            print(f'writing: {asset}')
            dso.to_zarr(asset, mode="w", consolidated=True)
            dc.fnl_make_cache(
                experiment,
                component,
                stream,
                member_id,
                variable,
                f"drift-corrected.{operation}",
                add_ops,
            )

In [ ]:
del client
del cluster

In [ ]:
collection = 'drift-corrected.ann.ts-glb'
sub_spec = dict(
    name=collection,
    experiment=['20C', 'RCP85'],
    member_id=dc.ocean_bgc_member_ids,
)

catalog = funnel.to_intake_esm(agg_member_id=True).search(**sub_spec)

cat = catalog.search(variable=variable_list)
dsets = cat.to_dataset_dict()

exp_keys = [
    f'20C.ocn.pop.h.{collection}',
    f'RCP85.ocn.pop.h.{collection}',
]

ds = xr.concat([dsets[k] for k in exp_keys], dim='time', coords='minimal', compat='override')
ds = ds.compute()
ds

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(6, 8), squeeze=False)

ax = axs[0, 0]
for member_id in ds.member_id.values:
    ax.plot(
        ds.time,
        ds.TEMP.sel(member_id=member_id),
        linestyle='-',
        color='gray',
        linewidth=0.5,
    )

ax.plot(ds.time, ds.TEMP.mean('member_id'), '-', color='k', linewidth=2)
ax.set_title('Global mean upper ocean (z > -1000 m) temperature')

ax = axs[1, 0]
for member_id in ds.member_id.values:
    ax.plot(
        ds.time,
        ds.pO2.sel(member_id=member_id),
        linestyle='-',
        color='gray',
        linewidth=0.5,
    )

ax.plot(ds.time, ds.pO2.mean('member_id'), '-', color='k', linewidth=2)
ax.set_title(r'Global mean upper ocean (z > -1000 m) $P_{\mathrm{O}_2}$');

In [ ]:
curator = util.curator_local_assets()

clobber = True

this_notebook = 'compute-ts-and-epoch-means.ipynb'

curator = util.curator_local_assets()
key = 'cesm-le-global-ts'
if clobber:
    cache_file = f'data/cache/{key}.zarr'
    os.makedirs(os.path.dirname(cache_file), exist_ok=True)
    ds.to_zarr(cache_file, mode='w', consolidated=True)

    curator.add_source(
        key=key,
        urlpath=cache_file,
        description=f'CESM-LE global means (upper 1 km) computed by {this_notebook}',
        driver='zarr',
        overwrite=True,
    )

cat = curator.open_catalog()
ds_cache = cat[key].to_dask()
xr.testing.assert_identical(ds, ds_cache)

In [ ]:
sub_spec = dict(
    name='drift-corrected.ann',
    experiment=['20C', 'RCP85'],
    member_id=dc.ocean_bgc_member_ids,
)

catalog = funnel.to_intake_esm(agg_member_id=True).search(**sub_spec)

cat = catalog.search(variable=variable_list)
dsets = cat.to_dataset_dict()

In [ ]:
epoch = xr.DataArray(['ref_climate', '2100_climate'], dims=('epoch'))
with xr.set_options(keep_attrs=True):
    ds = xr.concat(
        [
            dsets['20C.ocn.pop.h.drift-corrected.ann'].sel(time=slice(1920, 1965)).mean('time'),
            dsets['RCP85.ocn.pop.h.drift-corrected.ann'].sel(time=slice(2080, 2100)).mean('time'),
        ],
        dim=epoch,
        coords='minimal',
    )
ds['REGION_MASK'] = ds.REGION_MASK[0, :, :]
ds

In [ ]:
vol_mask = ops.pop_ocean_volume(ds)

with xr.set_options(keep_attrs=True):
    ds_glb = ds.weighted(ds.dz).mean(['z_t']).compute()
    ds_glb = ds_glb.drop(['REGION_MASK'])
    for v in ds_glb.data_vars:
        ds_glb[v] = ds_glb[v].mean('member_id')
ds_glb

In [ ]:
for v, e in product(ds_glb.data_vars, ds_glb.epoch.values):
    plt.figure()
    ds_glb[v].sel(epoch=e).plot();

In [ ]:
curator = util.curator_local_assets()

clobber = True

this_notebook = 'compute-ts-and-epoch-means.ipynb'

curator = util.curator_local_assets()
key = 'cesm-le-epoch-means'
if clobber:
    cache_file = f'data/cache/{key}.zarr'
    os.makedirs(os.path.dirname(cache_file), exist_ok=True)
    ds_glb.to_zarr(cache_file, mode='w', consolidated=True)

    curator.add_source(
        key=key,
        urlpath=cache_file,
        description=f'CESM-LE epoch means (upper 1 km) computed by {this_notebook}',
        driver='zarr',
        overwrite=True,
    )

cat = curator.open_catalog()
ds_cache = cat[key].to_dask()
xr.testing.assert_identical(ds_glb, ds_cache)